In [52]:
# !pip install pysimplegui
#pip install --force-reinstall -v "pyabf==2.3.8" #delite and reinstall the certain version
#pip install -Iv MySQL_python==1.2.2 # install the certain version
#!pip install matplotx==0.0.0

#import abf_explorer
#!pip install --upgrade numpy
import pyabf

import numpy as np

import matplotlib.pyplot as plt
#import matplotx

import statistics

import PySimpleGUI as sg

import pandas as pd

import os
from scipy.signal import find_peaks
from scipy.ndimage import gaussian_filter1d

In [10]:
my_file = pyabf.ABF("2023_08_03_0054.abf")

In [ ]:
# GUI to choose file
tres = 0
my_file = False
while True:
    event, values = sg.Window('Get filename example', [[sg.Text('Filename')], [sg.Input(), sg.FileBrowse()], [sg.OK(), sg.Cancel()] ]).read(close=True)
    if event == 'Cancel':
        break
    try:
        my_file = pyabf.ABF(values[0])
        break
    except:
        print('Incorrect file format')

In [ ]:
# Response of both cells plots

plt.figure(figsize=(40,20))

plt.subplot (1,2,1)

for my_sweepNumber in my_file.sweepList:
    my_file.setSweep(sweepNumber = my_sweepNumber, channel = 0)
    i1, i2 = 0, int(my_file.dataRate * 1)  # plot part of the sweep
    dataX = my_file.sweepX[i1:i2] + .025 * my_sweepNumber
    dataY = my_file.sweepY[i1:i2] + 15 * my_sweepNumber
    plt.plot(dataX, dataY, color='C0', alpha=.5, label=my_sweepNumber)
# matplotx.line_labels()
plt.gca().axis('off')  # hide axes to enhance floating effect
plt.title("Response of Channel 0")

plt.subplot (1,2,2)

for my_sweepNumber in my_file.sweepList:
    my_file.setSweep(sweepNumber = my_sweepNumber, channel = 2)
    i1, i2 = int(my_file.dataRate * 1.5), int(my_file.dataRate * 2.5)  # plot part of the sweep #15000 in original for 10kHz sampling
    dataX = my_file.sweepX[i1:i2] + .025 * my_sweepNumber
    dataY = my_file.sweepY[i1:i2] + 15 * my_sweepNumber
    plt.plot(dataX, dataY, color='C0', alpha=.5, label=my_sweepNumber)
# matplotx.line_labels()
plt.gca().axis('off')  # hide axes to enhance floating effect
plt.title("Response of Channel 2")


plt.show()

In [ ]:
# Choose channel for analysis 

my_search_ch = int(input("Please enter your #channel (0 or 2!) "))

In [49]:
num_points = 40 

#define the area of interest for the cell in 1st and 2nd devices 
def TimePeriodBorders (my_file_abf):
    sample_rate = int(my_file_abf.dataRate)
    if sample_rate == 20000: 
        # Time for PreStim and Stim for Channel 0 
        my_start10 = 0 
        my_end10 = 2800    
        my_start11 = 4000 
        my_end11 = 12800 
        # Time for PreStim and Stim for Channel 2
        my_start20 = 28000 
        my_end20 = 30800 
        my_start21 = 32000 
        my_end21 = 40800 
    elif sample_rate == 10000:
        # Time for PreStim and Stim for Channel 0
        my_start10 = 0 
        my_end10 = 1400     
        my_start11 = 2000 
        my_end11 = 6400 
        # Time for PreStim and Stim for Channel 2
        my_start20 = 14000
        my_end20 = 15400
        my_start21 = 16000
        my_end21 = 20000 
    else:
        print("Check the Sampling rate!")
        return ([],[],[],[],[],[],[],[])
        
    return (my_start10, my_end10, 
            my_start11, my_end11, 
            my_start20, my_start21, 
            my_start21, my_end21)



#find the last sweep with no APs    
def sweep_AP (my_file_abf, search_ch, start, stop):
    #choose the sweep with at least 1 spikes
    for sweep_num in range(len(my_file_abf.sweepList)):
        my_file_abf.setSweep(sweep_num, channel = search_ch)
        peaks = find_peaks(my_file_abf.sweepY[start:stop], height=0)
        #print(peaks, 'My peaks')
        if len(peaks[0])>=1:
            sweep = sweep_num #sweep is detected
            break
        elif len(peaks[0])<1 and sweep_num==len(my_file_abf.sweepList):
            print('Sweep is not detected!')
            sweep = None
    print(sweep)
    return sweep



#relative values of I and V and Rin
def IV_relative (my_file_abf, my_sweep_list,
                 voltage_channel, current_channel,
                 pre_start, pre_end, 
                 post_start, post_end, 
                 currents_delta_list = [], voltages_delta_list = [], voltage_range_ind_list = [],
                 Rin_relative_list = [], Rin_relative_norm_list = []):
    
        for my_sweepNumber in range(my_sweep_list):
            
            my_file_abf.setSweep(sweepNumber = my_sweepNumber, channel = voltage_channel)
            if voltage_channel==0:
                voltage_pre_stim = np.mean(my_file_abf.sweepY[pre_start: pre_end])
                voltage_dur_stim = np.mean(my_file_abf.sweepY[post_start: post_end])
            elif voltage_channel==2:
                voltage_pre_stim = np.mean(my_file_abf.sweepY[pre_start: pre_end])
                voltage_dur_stim = np.mean(my_file_abf.sweepY[post_start: post_end])
                
            if voltage_dur_stim >= -80 and voltage_dur_stim <= -50:
                voltage_range_ind_list.append(my_sweepNumber)

            voltage_delta = voltage_dur_stim - voltage_pre_stim
            voltages_delta_list.append(voltage_delta)
            
            my_file_abf.setSweep(sweepNumber = my_sweepNumber, channel = current_channel)
            if current_channel==1:
                current_pre_stim = np.mean(my_file_abf.sweepY[pre_start: pre_end])
                current_dur_stim = np.mean(my_file_abf.sweepY[post_start: post_end])
            elif current_channel==3:
                current_pre_stim = np.mean(my_file_abf.sweepY[pre_start: pre_end])
                current_dur_stim = np.mean(my_file_abf.sweepY[post_start: post_end])
                
            current_delta = current_dur_stim - current_pre_stim
            currents_delta_list.append(current_delta)  
            
        Rin_relative_list = [x / y for x,y in zip(voltages_delta_list, currents_delta_list)]
        Rin_relative_norm_list = np.round(Rin_relative_list/max(Rin_relative_list), 2)
        Rin_from_range = [Rin_relative_list[x] for x in voltage_range_ind_list]
        Rin_property = np.mean(Rin_from_range)
            
        return currents_delta_list, voltages_delta_list, Rin_relative_list, Rin_relative_norm_list, Rin_from_range, Rin_property
    
    

def sweep_3Aps (my_file_abf, search_ch, start, stop):
    #choose the sweep with at least 3 spikes
    for sweep_num in range(len(my_file_abf.sweepList)):
        my_file_abf.setSweep(sweep_num, channel = search_ch)
        peaks = find_peaks(my_file_abf.sweepY[start:stop], height=0)
        #print(peaks, 'My peaks')
        if len(peaks[0])>=3:
            sweep = sweep_num #sweep is detected
            break
        elif len(peaks[0])<3 and sweep_num==len(my_file_abf.sweepList):
            print('Sweep is not detected!')
            sweep = None
    #print(sweep)
    return sweep



freq = 1/my_file.dataRate
num_points = 40 #number of points for smoothing

def AP_search_time (my_file_abf, search_ch):
    if search_ch == 0:
        start = int(0.14*my_file_abf.dataRate)
        stop = int(0.7*my_file_abf.dataRate)
    if search_ch == 2:
        start = int(1.54*my_file_abf.dataRate)
        stop = int(2.1*my_file_abf.dataRate)
    return start, stop


#inflection points detection
def infl_points (my_file_abf, sweep, search_ch, start, stop, num_pts):
        my_file_abf.setSweep(sweep, channel = search_ch)
        smooth = gaussian_filter1d(my_file_abf.sweepY[start:stop], num_pts) # smooth
        smooth_d2 = np.gradient(np.gradient(smooth)) # compute second derivative
        infls = np.where(np.diff(np.sign(smooth_d2)))[0]# find switching points
        ind_infls = infls + start
        return ind_infls
    


#find indexes of peaks
def peak (my_file_abf, start, stop, v_level= -10):
    peaks= find_peaks(my_file_abf.sweepY[start:stop], height=v_level, distance=10)
    ind_peaks = peaks[0] + start
    return ind_peaks



def spike_begin (ind_peaks, ind_infls):
    #find the beginning of spike (the index before the peak index)
    start_ap_ind_list = [j for i in range (len(ind_peaks)) for j in range (len(ind_infls)) if ind_peaks[i]>ind_infls[j] and ind_peaks[i]<ind_infls[j+1]]
    start_ap_ind = ind_infls[start_ap_ind_list]
    #find the "end" of spike (the index after the peak index) - needs only for the FWHM calculation
    end_ap_ind_list = [j for i in range (len(ind_peaks)) for j in range (len(ind_infls)) if ind_peaks[i]<ind_infls[j] and ind_peaks[i]>ind_infls[j-1]]
    end_ap_ind = ind_infls[end_ap_ind_list]
    return start_ap_ind, end_ap_ind

In [50]:
time_points = TimePeriodBorders(my_file)
last_no_AP_sweep = sweep_AP(my_file, 0, time_points[0], time_points[3])
my_relative_values = IV_relative(my_file, last_no_AP_sweep, 0, 1, time_points[0], time_points[1], time_points[2], time_points[3], [], [], [], [], [])
print(my_relative_values)

7
([-80.856, -50.813408, -20.707605, 9.9725485, 39.878113, 70.633286, 100.96227], [-18.760635, -12.210091, -5.275528, 3.4816437, 13.551872, 22.78759, 30.623356], [0.23202527, 0.2402927, 0.25476283, 0.34912276, 0.33983234, 0.3226183, 0.30331483], array([0.66, 0.69, 0.73, ..., 0.97, 0.92, 0.87], dtype=float32), [0.25476283, 0.34912276, 0.33983234], 0.31457266)


In [55]:


#approximation to find exact position of FWHM (to lazy to put it to the function)

my_start, my_stop = AP_search_time(my_file, my_search_ch)
my_sweep = sweep_3Aps(my_file, my_search_ch, my_start, my_stop)
my_ind_infls = infl_points (my_file, my_sweep, my_search_ch, my_start, my_stop, num_points)
my_ind_peaks = peak (my_file, my_start, my_stop)
my_start_ap_ind = spike_begin(my_ind_peaks, my_ind_infls)[0]
my_end_ap_ind = spike_begin(my_ind_peaks, my_ind_infls)[1]



ap1_ampl = my_file.sweepY[my_ind_peaks[0]] - my_file.sweepY[my_start_ap_ind[0]] #mV
ap2_ampl = my_file.sweepY[my_ind_peaks[1]] - my_file.sweepY[my_start_ap_ind[1]] #mV
ap_ampl_ratio = np.round(ap2_ampl/ap1_ampl, 2)

half_ampl = my_file.sweepY[my_start_ap_ind[0]]+ap1_ampl/2
search_up = my_file.sweepY[my_start_ap_ind[0]:my_ind_peaks[0]]
search_up_ind = []
search_down = my_file.sweepY[my_ind_peaks[0]:my_start_ap_ind[1]]
search_down_ind = []

find_up_ind = np.searchsorted(search_up, half_ampl)
#print(find_up_ind)
if my_file.sweepY[my_start_ap_ind[0]+find_up_ind]>=half_ampl:
    search_up_ind = [find_up_ind-1, find_up_ind]
if my_file.sweepY[my_start_ap_ind[0]+find_up_ind]<half_ampl:
    search_up_ind = [find_up_ind, find_up_ind+1]
    
#print(search_up_ind, 'x indexes')
#print(my_file.sweepY[search_up_ind+my_start_ap_ind[0]], 'Y values')

#print(my_file.sweepX[search_up_ind+my_start_ap_ind[0]], 'time x for roots')
my_fit_up = np.polyfit(my_file.sweepX[search_up_ind+my_start_ap_ind[0]], my_file.sweepY[search_up_ind+my_start_ap_ind[0]], 1)
#print(my_fit)
up_p = [my_fit_up[0], my_fit_up[1]-half_ampl]
#print(up_p)
my_root_up = np.roots(up_p)
#print(my_root_up)


#print()
#print()

find_down_ind = np.searchsorted(-search_down, -half_ampl) #not the best way due to criteria for absending order 

#print(find_up_ind)
if my_file.sweepY[my_ind_peaks[0]+find_down_ind]>=half_ampl:
    search_down_ind = [find_down_ind, find_down_ind+1]
if my_file.sweepY[my_ind_peaks[0]+find_down_ind]<half_ampl:
    search_down_ind = [find_down_ind-1, find_down_ind]
    
#print(search_down_ind, 'x indexes')
#print(my_file.sweepY[search_down_ind+my_ind_peaks[0]], 'Y values')

#print(my_file.sweepX[search_down_ind+my_ind_peaks[0]], 'time x for roots')
my_fit_down = np.polyfit(my_file.sweepX[search_down_ind+my_ind_peaks[0]], my_file.sweepY[search_down_ind+my_ind_peaks[0]], 1)
#print(my_fit_down, 'coef') #coef
down_p = [my_fit_down[0], my_fit_down[1]-half_ampl]
#print(down_p)
my_root_down = np.roots(down_p)
#print(my_root_down)

#print()
#print(my_ind_peaks[0])
#print(my_root_up+my_start_ap_ind[0]+search_up_ind[0])
#print(my_root_down+my_ind_peaks[0]+search_down_ind[0])
my_half_total = my_root_down-my_root_up
my_fwhm_approximated = np.round(my_half_total*1000, 2)
#print()
print(my_fwhm_approximated)

(array([], dtype=int64), {'peak_heights': array([], dtype=float64)}) My peaks
(array([], dtype=int64), {'peak_heights': array([], dtype=float64)}) My peaks
(array([], dtype=int64), {'peak_heights': array([], dtype=float64)}) My peaks
(array([], dtype=int64), {'peak_heights': array([], dtype=float64)}) My peaks
(array([], dtype=int64), {'peak_heights': array([], dtype=float64)}) My peaks
(array([], dtype=int64), {'peak_heights': array([], dtype=float64)}) My peaks
(array([], dtype=int64), {'peak_heights': array([], dtype=float64)}) My peaks
(array([1813, 2144, 2593], dtype=int64), {'peak_heights': array([19.7144, 30.9448, 28.3813])}) My peaks
7
[2.14]
